# How obsimflow works and its architecture/design

In [1]:
import simpy

# Patient arrivals
There are two main patient arrival schemes:

- stationary Poisson arrivals
- scheduled arrivals

Stationary Poisson arrival generators are for modeling patients arriving due to spontaneous labor as well as for other non-scheduled patients requiring use of one or more patient care units (e.g. observation or post-partum).

Scheduled arrival generators can be used to model scheduled C-section and scheduled induction patient arrivals.

### Poisson arrivals

The `PatientPoissonArrivals` class generates `Patient` objects
according to a stationary poisson process with a specified
rate. In addition to the mean arrival rate, the arrival generator
is initialized with a unique arrival stream identifier (`str`), and
a numpy random number generator (`numpy.random.default_rng`) whose
seed is specified in the simulation scenario config file. There
are two ways to control the stopping of patient generation.

- by time via setting `stop_time` (default is `simpy.core.Infinity`)
- by number of arrivals via setting ``max_arrivals` (default is `simpy.core.Infinity`)

The last input parameter is a `PatientFlowSystem` object. This object is the primary "container" for components making up the system to be simulated. It is this object that will consume the generated arrival objects.

Within SimPy, modeling Poisson arrivals can be done with a loop that keeps generating `Timeout` events with exponential inter-event times until one of the stopping conditions is met.

There is a single state related attribute, `num_patients_created`, that is incremented on each arrival event.

After the interarrival `Timeout` event is processed:

- a unique arrival id (`str`) is created based on the arrival stream id and the value of `num_patients_created`,
- a new instance of `Patient` is created. In DES terms, each instance is a new *entity* that will flow through the system.

To create a new `Patient` instance we pass the unique arrival id, the arrival stream id, the current simulation time, and the `PatientFlowSystem` object to the `Patient.__init__()` method. The `Patient` object will handle inserting the new patient into the system at the `ENTRY` location.

#### Possible improvements

As implemented, the `PatientPoissonArrivals` class is **not** a general Poisson arrival generator object that can be reused without modifying the source code. It is tightly coupled to both the type of entity object being created (`Patient`) and the system object to which the entity arrives (`PatientFlowSystem`). The current design is transparent and was easy to create, but we should be able to come up with a way for a general Poisson arrival object to broadcast a signal that a new arrival has occured and that some other object (i.e. the system object) can detect and respond. The signal would need to contain all relevant information needed by the system object to create the new entity instance and route it into the flow system.

In [ ]:
class PatientPoissonArrivals:
    """ Generates patients according to a stationary Poisson process with specified rate.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        arrival_stream_uid : ArrivalType
            unique name of random arrival stream
        arrival_rate : float
            Poisson arrival rate (expected number of arrivals per unit time)
        arrival_stream_rg : numpy.random.Generator
            used for interarrival time generation
        stop_time : float
            Stops generation at the stoptime. (default Infinity)
        max_arrivals : int
            Stops generation after max_arrivals. (default Infinity)
        patient_flow_system : (default None) PatientFlowSystem into which the arrival is inserted
            This allows us to kick off a patient flowing through the system. If omitted,
            no patient instances are actually created.

        TODO: Decouple the PP generator from the actual creation of model specific entities
    """

    def __init__(self, env, arrival_stream_uid: ArrivalType, arrival_rate: float, arrival_stream_rg,
                 stop_time=simpy.core.Infinity, max_arrivals=simpy.core.Infinity,
                 patient_flow_system=None):

        # Parameter attributes
        self.env = env
        self.arrival_stream_uid = arrival_stream_uid
        self.arr_rate = arrival_rate
        self.arr_stream_rg = arrival_stream_rg
        self.stop_time = stop_time
        self.max_arrivals = max_arrivals
        self.patient_flow_system = patient_flow_system

        # State attributes
        self.num_patients_created = 0

        # Trigger the run() method and register it as a SimPy process
        env.process(self.run())

    def run(self):
        """
        Generate entities until stopping condition met
        """

        # Main entity creation loop that terminates when stoptime reached
        while self.env.now < self.stop_time and \
                self.num_patients_created < self.max_arrivals:
            # Compute next interarrival time
            iat = self.arr_stream_rg.exponential(1.0 / self.arr_rate)
            # Delay until time for next arrival
            yield self.env.timeout(iat)
            self.num_patients_created += 1

            new_entity_id = f'{self.arrival_stream_uid}_{self.num_patients_created}'

            if self.patient_flow_system is not None:
                new_patient = Patient(new_entity_id, self.arrival_stream_uid,
                                      self.env.now, self.patient_flow_system)

                logging.debug(
                    f"{self.env.now:.4f}: {new_patient.patient_id} created at {self.env.now:.4f} ({self.patient_flow_system.sim_calendar.now()}).")